In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import re

# load data

In [2]:
all_abstracts = pd.read_csv('data/included_abstracts.csv', index_col = 0)

all_abstracts['article_date'] = pd.to_datetime(all_abstracts['article_date'])
decade_df = all_abstracts[(all_abstracts['article_date'] > '2012-01-01') & (all_abstracts['article_date'] <'2022-01-01')]
len(decade_df)

28703

In [3]:
methods_df = pd.read_csv('output/methods_scored.csv', index_col = 0)
print(len(methods_df))
methods_df.info()

9071
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9071 entries, 0 to 9070
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pmid                 9071 non-null   int64  
 1   title                9071 non-null   object 
 2   methods              9071 non-null   object 
 3   sample_answer        9071 non-null   object 
 4   sample_score         8648 non-null   float64
 5   database_answer      9071 non-null   object 
 6   database_score       8648 non-null   float64
 7   organisation_answer  9071 non-null   object 
 8   organisation_score   8648 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 708.7+ KB


In [4]:
abstracts_df = pd.read_csv('output/abstracts_scored.csv', index_col = 0)
print(len(abstracts_df))
abstracts_df.info()

28703
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28703 entries, 0 to 28702
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pmid                 28703 non-null  int64  
 1   title                28702 non-null  object 
 2   abstract             28703 non-null  object 
 3   disease_answer       28703 non-null  object 
 4   disease_score        28703 non-null  float64
 5   sample_answer        28703 non-null  object 
 6   sample_score         28703 non-null  float64
 7   modality_answer      28703 non-null  object 
 8   modality_score       28703 non-null  float64
 9   database_answer      28703 non-null  object 
 10  database_score       28703 non-null  float64
 11  organisation_answer  28703 non-null  object 
 12  organisation_score   28703 non-null  float64
dtypes: float64(5), int64(1), object(7)
memory usage: 3.1+ MB


# abstract sample sizes

In [5]:
# all letters to lower case
abstracts_df['sample_answer'] = abstracts_df['sample_answer'].map(lambda x:x.lower() if type(x) == str else x) #reduce all to lowercase

In [6]:
# convert text numbers to numeric

from text_to_num import alpha2digit

templist = []

for x in abstracts_df['sample_answer']:
    try:
        templist.append(alpha2digit(x, "en"))
    except Exception as e:
        print(e)
        templist.append('skip')

abstracts_df['sample_answer'] = templist

In [7]:
# remove commas (no spaces)
abstracts_df['sample_answer']=abstracts_df['sample_answer'].str.replace(',','')

In [8]:
# remove entries with no numeric characters
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

templist2 = []

for x in abstracts_df['sample_answer']:
    if has_numbers(x) == True:
        templist2.append(x)
    else:
        templist2.append(np.nan)

abstracts_df['sample_answer'] = templist2        

In [9]:
#abstracts_df['sample_answer'].sample(15)

# methods sample sizes

In [10]:
# all letters to lower case
methods_df['sample_answer'] = methods_df['sample_answer'].map(lambda x:x.lower() if type(x) == str else x) #reduce all to lowercase

In [11]:
# convert text numbers to numeric

from text_to_num import alpha2digit

templist = []

for x in methods_df['sample_answer']:
    try:
        templist.append(alpha2digit(x, "en"))
    except Exception as e:
        print(e)
        templist.append('skip')

methods_df['sample_answer'] = templist

In [12]:
# remove commas (no spaces)
methods_df['sample_answer']=methods_df['sample_answer'].str.replace(',','')

In [13]:
templist2 = []

for x in methods_df['sample_answer']:
    if has_numbers(x) == True:
        templist2.append(x)
    else:
        templist2.append(np.nan)

methods_df['sample_answer'] = templist2 

In [14]:
#methods_df.sample(15)

# sample size sheet and combined truth column for annotation

In [15]:
abstract_sizes = abstracts_df[['abstract', 'pmid', 'sample_answer', 'sample_score']].copy()
abstract_sizes.rename(columns={"sample_answer": "abstract_size", "sample_score": "abstract_score"}, inplace=True)

method_sizes = methods_df[['methods', 'pmid', 'sample_answer', 'sample_score']].copy()
method_sizes.rename(columns={"sample_answer": "method_size", "sample_score": "method_score"}, inplace=True)

In [16]:
all_sizes = abstract_sizes.merge(method_sizes, on = 'pmid', how = 'left')
all_sizes = all_sizes[['pmid', 'abstract', 'methods', 'abstract_size', 'abstract_score',
                       'method_size', 'method_score']].astype('object')

In [17]:
## rules for generating truth column
# (1) keep only absolute numerics from abstract / method sizes
# (2) keep all method derived sizes
# (3) where method probability is below threshold, AND if the abstract size is larger, drop the method size
# (4) where method = NaN, replace with abstract derived size

In [18]:
templist = []

for x in all_sizes['abstract_size']:
    try:
        if x.isdigit() == True:
            templist.append(x)
        else:
            templist.append(np.nan)
    except:
        templist.append(np.nan)
        
all_sizes['abstract_truth'] = templist

#check if only contains numbers in method

templist = []

for x in all_sizes['method_size']:
    try:
        if x.isdigit() == True:
            templist.append(x)
        else:
            templist.append(np.nan)
    except:
        templist.append(np.nan)
        
all_sizes['method_truth'] = templist

all_sizes[['abstract_truth', 'method_truth']] = all_sizes[['abstract_truth', 'method_truth']].apply(pd.to_numeric)

In [19]:
templist = []

for i, row in all_sizes.iterrows():
    if row['method_score'] < 0.0005 and row['abstract_truth'] > row['method_truth']:
        templist.append(np.nan)
    else:
        templist.append(row['method_truth'])

all_sizes['method_truth'] = templist

In [20]:
all_sizes = all_sizes.assign(size_truth = all_sizes['method_truth'].fillna(all_sizes['abstract_truth'])).drop(['method_truth', 'abstract_truth'], axis=1)

In [21]:
all_sizes.sample(50)

,pmid,abstract,methods,abstract_size,abstract_score,method_size,method_score,size_truth
21466,29992732,Chest wall toxicity is observed after stereota...,NaN,25,0.658893,NaN,NaN,25.0
12733,30906246,Do men and women have different brains? Previo...,NaN,1065,0.388785,NaN,NaN,1065.0
13175,31026030,Obesity is a leading cause of high health care...,Data Sources The Strengthening the Reporting...,3138,0.282362,30,0.002805,30.0
10056,34330087,"Brain arteriolosclerosis, one of the main path...",NaN,119,0.807792,NaN,NaN,119.0
18361,34346318,This paper introduces methods to estimate aspe...,NaN,32,0.686557,NaN,NaN,32.0
16865,34451921,This study aimed to investigate the effects of...,NaN,142,0.676577,NaN,NaN,142.0
14722,31391148,We performed a transcriptome-wide meta-analysi...,NaN,19,0.178776,NaN,NaN,19.0
19776,33660087,"In this work, an attempt has been made to clas...",NaN,NaN,0.013159,NaN,NaN,NaN
5533,28234631,Diffusion tensor imaging (DTI) is a promising ...,NaN,150,0.583266,NaN,NaN,150.0
383,34296379,Although type 2 diabetes (T2D) remission after...,NaN,5,0.38656,NaN,NaN,5.0


In [22]:
all_sizes.to_csv('output/_annotation_sizes.csv')